In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
!pip install azure-storage-blob
!pip install boto3
!pip install google-cloud-storage

In [ ]:
#import libraries

import pandas as pd
import numpy as np
import json
import requests
import boto3
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
from google.cloud import storage
from io import StringIO

S1 --> Gather --> decompress --> convert to tabular --> clean --> reformat --> consolidate --> transform --> load

In [ ]:
df_raw = pd.read_excel(r'/content/gdrive/My Drive/Queens Rolling Sales 2022/Queens_Rolling_2022.xlsx')
df_raw.head()
print(df_raw.columns)

Index(['BOROUGH', 'NEIGHBORHOOD', 'BUILDING CLASS CATEGORY',
       'TAX CLASS AT PRESENT', 'BLOCK', 'LOT', 'EASE-MENT',
       'BUILDING CLASS AT PRESENT', 'ADDRESS', 'APARTMENT NUMBER', 'ZIP CODE',
       'RESIDENTIAL\nUNITS', 'COMMERCIAL\nUNITS', 'TOTAL \nUNITS',
       'LAND \nSQUARE FEET', 'GROSS \nSQUARE FEET', 'YEAR BUILT',
       'TAX CLASS AT TIME OF SALE', 'BUILDING CLASS\nAT TIME OF SALE',
       'SALE PRICE', 'SALE DATE', 'Price per Square foot',
       'Mean per Square foot', 'Squared Differences from Mean',
       'Variance per Square foot'],
      dtype='object')


In [ ]:
display(pd.DataFrame(df_raw))

,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,BLOCK,LOT,EASE-MENT,BUILDING CLASS AT PRESENT,ADDRESS,APARTMENT NUMBER,...,GROSS \nSQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS\nAT TIME OF SALE,SALE PRICE,SALE DATE,Price per Square foot,Mean per Square foot,Squared Differences from Mean,Variance per Square foot
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
1,4.0,AIRPORT LA GUARDIA,01 ONE FAMILY DWELLINGS,1,949.0,34.0,NaN,A5,19-48 81ST STREET,NaN,...,1224.0,1945.0,1.0,A5,864195.0,2022-07-15,706.042,NaN,NaN,NaN
2,4.0,AIRPORT LA GUARDIA,01 ONE FAMILY DWELLINGS,1,949.0,37.0,NaN,A5,19-54 81ST STREET,NaN,...,1224.0,1945.0,1.0,A5,815000.0,2022-06-24,338.809,NaN,NaN,NaN
3,4.0,AIRPORT LA GUARDIA,01 ONE FAMILY DWELLINGS,1,976.0,55.0,NaN,A5,21-33 80TH STREET,NaN,...,1224.0,1950.0,1.0,A5,830000.0,2022-09-23,320.724,NaN,NaN,NaN
4,4.0,AIRPORT LA GUARDIA,01 ONE FAMILY DWELLINGS,1,976.0,60.0,NaN,A5,21-23 80TH STREET,NaN,...,1224.0,1950.0,1.0,A5,900000.0,2022-02-08,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28836,4.0,WOODSIDE,44 CONDO PARKING,4,1337.0,1136.0,NaN,RG,"40-53 62ND STREET, P8",P8,...,NaN,2005.0,4.0,RG,640000.0,2022-07-12,#DIV/0!,NaN,NaN,NaN
28837,4.0,WOODSIDE,44 CONDO PARKING,4,2324.0,1132.0,NaN,RG,"63-14 QUEENS BOULEVARD, P-27",P-27,...,NaN,2008.0,4.0,RG,645000.0,2022-03-21,#DIV/0!,NaN,NaN,NaN
28838,4.0,WOODSIDE,44 CONDO PARKING,4,2324.0,1136.0,NaN,RG,"63-14 QUEENS BOULEVARD, P-31",P-31,...,NaN,2008.0,4.0,RG,760000.0,2022-05-06,#DIV/0!,NaN,NaN,NaN
28839,4.0,WOODSIDE,44 CONDO PARKING,4,2324.0,1144.0,NaN,RG,"63-14 QUEENS BOULEVARD, P-39",P-39,...,NaN,2008.0,4.0,RG,28000.0,2022-03-31,#DIV/0!,NaN,NaN,NaN


In [ ]:
#data types
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28841 entries, 0 to 28840
Data columns (total 25 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   BOROUGH                         28840 non-null  float64       
 1   NEIGHBORHOOD                    28840 non-null  object        
 2   BUILDING CLASS CATEGORY         28840 non-null  object        
 3   TAX CLASS AT PRESENT            28835 non-null  object        
 4   BLOCK                           28840 non-null  float64       
 5   LOT                             28840 non-null  float64       
 6   EASE-MENT                       0 non-null      float64       
 7   BUILDING CLASS AT PRESENT       28835 non-null  object        
 8   ADDRESS                         28840 non-null  object        
 9   APARTMENT NUMBER                4763 non-null   object        
 10  ZIP CODE                        28839 non-null  float64       
 11  RE

In [ ]:
df_cleaned = df_raw.copy()

In [ ]:
df_cleaned = df_raw.drop(columns = ['EASE-MENT'])
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28841 entries, 0 to 28840
Data columns (total 24 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   BOROUGH                         28840 non-null  float64       
 1   NEIGHBORHOOD                    28840 non-null  object        
 2   BUILDING CLASS CATEGORY         28840 non-null  object        
 3   TAX CLASS AT PRESENT            28835 non-null  object        
 4   BLOCK                           28840 non-null  float64       
 5   LOT                             28840 non-null  float64       
 6   BUILDING CLASS AT PRESENT       28835 non-null  object        
 7   ADDRESS                         28840 non-null  object        
 8   APARTMENT NUMBER                4763 non-null   object        
 9   ZIP CODE                        28839 non-null  float64       
 10  RESIDENTIAL
UNITS               22469 non-null  float64       
 11  CO

In [ ]:
#specify the path to your JSON configuration file
config_file_path = "config.json"

#load with JSON configuration file
with open(config_file_path, 'r') as config_file:
    config = json.load(config_file)

#print the configuration
#Connection_STRING = config["connectionString"]

CONNECTION_STRING_AZURE_STORAGE = config["connectionString"]
CONTAINER_AZURE = 'queensrollingsales'
blob_name = "queensrollingsales_20240402.csv"

#covert Dataframe to csv
output = StringIO()
df_raw.to_csv(output, index=False)
data = output.getvalue()
output.close()

#create the blobservice client object
blob_service_client = BlobServiceClient.from_connection_string(CONNECTION_STRING_AZURE_STORAGE)

#get a blob client using the container and blob name
blob_client = blob_service_client.get_blob_client(container=CONTAINER_AZURE,blob=blob_name)

#upload the csv data
blob_client.upload_blob(data, overwrite=True)

print(f"Uploaded {blob_name} to Azure Blob Storage in container {CONTAINER_AZURE}.")

Uploaded queensrollingsales_20240402.csv to Azure Blob Storage in container queensrollingsales.
